In [ ]:
# %% Cell 1: Enhanced Imports with Multi-Seed Analysis Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import Ridge, ElasticNet, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from scipy.optimize import differential_evolution
from scipy import stats
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

# MULTIPLE SEEDS FOR ROBUST ANALYSIS
SEEDS = [42, 123, 456, 789, 1001, 2024, 3141, 5555, 8888, 9999]

print("Enhanced ML Analysis for FacoDMEK Eyes - EXTENDED BOUNDS")
print("="*60)
print(f"Testing {len(SEEDS)} different random seeds for robust results")
print(f"Seeds: {SEEDS}")
print("="*60)
print("\nWARNING: Using extended parameter bounds")
print("This may lead to better results but also higher overfitting risk")
print("Multi-seed validation is crucial!")
print("="*60)

# Storage for results across seeds
all_results = []

In [ ]:
# %% Cell 2: Load Data with Proper Feature Engineering
# Load the Excel file
df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Cleaned Data')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

# Calculate derived features
df['K_avg_Kerato'] = (df['Keratometric Ks'] + df['Keratometric Kf']) / 2
df['K_Astigmatism_Kerato'] = df['Keratometric Ks'] - df['Keratometric Kf']
df['Post_Ant_Ratio'] = df['Posterior Km'] / df['Anterior Km']
df['AL_K_Ratio'] = df['Bio-AL'] / df['K_avg_Kerato']
df['K_Cylinder'] = np.abs(df['K_Astigmatism_Kerato'])
df['AL_Category'] = pd.cut(df['Bio-AL'], bins=[0, 22, 24.5, 26, 100], 
                           labels=['short', 'normal', 'long', 'very_long'])

print(f"\nFeature engineering completed")

# Prepare complete cases
required_cols = ['Bio-AL', 'K_avg_Kerato', 'IOL Power', 
                 'PostOP Spherical Equivalent', 'A-Constant']
df_complete = df[df[required_cols].notna().all(axis=1)].copy()

print(f"\nComplete cases for analysis: {len(df_complete)} out of {len(df)}")

In [ ]:
# %% Cell 3: Run Complete Analysis with EXTENDED PARAMETER BOUNDS
print("MULTI-SEED ROBUST ANALYSIS WITH EXTENDED BOUNDS")
print("="*60)

# Define SRK/T function
def calculate_SRKT(AL, K, A_const, nc=1.333):
    """Standard SRK/T formula"""
    if pd.isna(AL) or pd.isna(K) or pd.isna(A_const) or K <= 0 or AL <= 0:
        return np.nan
    
    try:
        na = 1.336
        r = 337.5 / K
        
        if AL > 24.2:
            LCOR = -3.446 + 1.716 * AL - 0.0237 * AL**2
        else:
            LCOR = AL
        
        Cw = -5.41 + 0.58412 * LCOR + 0.098 * K
        
        if r**2 - (Cw**2 / 4) < 0:
            return np.nan
        
        H = r - np.sqrt(r**2 - (Cw**2 / 4))
        ACDconst = 0.62467 * A_const - 68.747
        offset = ACDconst - 3.336
        ACDest = H + offset
        
        RETHICK = 0.65696 - 0.02029 * AL
        LOPT = AL + RETHICK
        
        ncm1 = nc - 1
        IOL = (1000 * na * (na * r - ncm1 * LOPT)) / ((LOPT - ACDest) * (na * r - ncm1 * ACDest))
        
        return IOL
    except:
        return np.nan

# Extended SRK/T with all parameters
def calculate_SRKT_ml(AL, K, A_const, params):
    """ML-optimized SRK/T formula with extended parameters"""
    nc = params[0]
    cw_const1, cw_const2, cw_const3 = params[1:4]
    acd_const1, acd_const2 = params[4:6]
    offset_const = params[6]
    rethick_const1, rethick_const2 = params[7:9]
    
    if pd.isna(AL) or pd.isna(K) or pd.isna(A_const) or K <= 0 or AL <= 0:
        return np.nan
    
    try:
        na = 1.336
        r = 337.5 / K
        
        if AL > 24.2:
            LCOR = -3.446 + 1.716 * AL - 0.0237 * AL**2
        else:
            LCOR = AL
        
        Cw = cw_const1 + cw_const2 * LCOR + cw_const3 * K
        
        if r**2 - (Cw**2 / 4) < 0:
            return np.nan
        
        H = r - np.sqrt(r**2 - (Cw**2 / 4))
        ACDconst = acd_const1 * A_const - acd_const2
        offset = ACDconst - offset_const
        ACDest = H + offset
        
        RETHICK = rethick_const1 - rethick_const2 * AL
        LOPT = AL + RETHICK
        
        ncm1 = nc - 1
        IOL = (1000 * na * (na * r - ncm1 * LOPT)) / ((LOPT - ACDest) * (na * r - ncm1 * ACDest))
        
        return IOL
    except:
        return np.nan

# Storage for seed-specific results
seed_results = []

# Run analysis for each seed
for seed_idx, SEED in enumerate(SEEDS):
    print(f"\n{'='*60}")
    print(f"ANALYZING SEED {seed_idx+1}/{len(SEEDS)}: {SEED}")
    print(f"{'='*60}")
    
    # Set seed
    np.random.seed(SEED)
    
    # Split data
    df_train_val, df_test = train_test_split(
        df_complete, 
        test_size=0.30,
        random_state=SEED,
        stratify=df_complete['AL_Category']
    )
    
    print(f"Train: {len(df_train_val)} eyes, Test: {len(df_test)} eyes")
    
    # Calculate baseline performance on training data
    df_train_val['SRKT_Baseline'] = df_train_val.apply(
        lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Kerato'], row['A-Constant']), 
        axis=1
    )
    df_train_val['Expected_SE'] = -(df_train_val['IOL Power'] - df_train_val['SRKT_Baseline'])
    df_train_val['SRKT_Error'] = df_train_val['PostOP Spherical Equivalent'] - df_train_val['Expected_SE']
    
    baseline_mae_train = df_train_val['SRKT_Error'].abs().mean()
    
    # Optimize nc using EXTENDED RANGE
    print("Optimizing nc with EXTENDED range...")
    df_opt_train = df_train_val[df_train_val['SRKT_Error'].notna()].copy()
    
    # EXTENDED NC RANGE: 1.325 to 1.345 (was 1.330 to 1.338)
    nc_range = np.arange(1.325, 1.345, 0.0002)
    best_nc = 1.333
    best_nc_mae = 999
    
    for nc in nc_range:
        predictions = df_opt_train.apply(
            lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Kerato'], 
                                     row['A-Constant'], nc=nc), 
            axis=1
        )
        expected_se = -(df_opt_train['IOL Power'] - predictions)
        errors = (df_opt_train['PostOP Spherical Equivalent'] - expected_se).abs()
        mae = errors.mean()
        
        if mae < best_nc_mae:
            best_nc_mae = mae
            best_nc = nc
    
    print(f"Optimal nc: {best_nc:.5f} (range tested: {nc_range[0]:.3f}-{nc_range[-1]:.3f})")
    
    # Optimize all parameters with EXTENDED BOUNDS
    print("Optimizing all parameters with EXTENDED bounds...")
    
    def objective_all_constants(params):
        errors_all = []
        for idx, row in df_opt_train.iterrows():
            pred = calculate_SRKT_ml(row['Bio-AL'], row['K_avg_Kerato'], 
                                    row['A-Constant'], params)
            if not np.isnan(pred):
                expected_se = -(row['IOL Power'] - pred)
                error = abs(row['PostOP Spherical Equivalent'] - expected_se)
                errors_all.append(error)
        
        if not errors_all:
            return 999
            
        mae = np.mean(errors_all)
        
        # STRONGER regularization for extended bounds
        standard_params = [1.333, -5.41, 0.58412, 0.098, 0.62467, 68.747, 3.336, 0.65696, 0.02029]
        penalty = 0
        for i, (p, s) in enumerate(zip(params, standard_params)):
            if i == 0:  # nc - moderate penalty
                penalty += 0.5 * ((p - s) / s) ** 2
            else:  # other parameters - stronger penalty
                penalty += 2.0 * ((p - s) / s) ** 2
        
        return mae + penalty
    
    # EXTENDED BOUNDS - much wider ranges
    bounds = [
        (1.325, 1.345),      # nc - extended from (1.330, 1.338)
        (-6.5, -4.3),        # cw_const1 - extended from (-5.6, -5.2)
        (0.50, 0.68),        # cw_const2 - extended from (0.56, 0.61)
        (0.070, 0.130),      # cw_const3 - extended from (0.093, 0.103)
        (0.55, 0.70),        # acd_const1 - extended from (0.61, 0.64)
        (60.0, 77.0),        # acd_const2 - extended from (67.0, 70.5)
        (2.5, 4.2),          # offset_const - extended from (3.2, 3.5)
        (0.55, 0.75),        # rethick_const1 - extended from (0.64, 0.67)
        (0.015, 0.025)       # rethick_const2 - extended from (0.019, 0.022)
    ]
    
    print("\nExtended bounds being used:")
    param_names = ['nc', 'cw_const1', 'cw_const2', 'cw_const3', 'acd_const1', 
                   'acd_const2', 'offset_const', 'rethick_const1', 'rethick_const2']
    for i, (name, (low, high)) in enumerate(zip(param_names, bounds)):
        print(f"  {name}: [{low:.3f}, {high:.3f}]")
    
    result_de = differential_evolution(
        objective_all_constants, 
        bounds,
        maxiter=50,      # More iterations for extended space
        popsize=20,      # Larger population
        seed=SEED,
        mutation=(0.5, 1.5),  # More exploration
        recombination=0.8,
        disp=False
    )
    
    optimal_params_all = result_de.x
    
    # Train ensemble with stronger regularization
    print("Training ensemble with regularization...")
    feature_cols = ['Bio-AL', 'K_avg_Kerato', 'A-Constant', 'K_Cylinder', 
                    'Post_Ant_Ratio', 'AL_K_Ratio', 'CCT']
    
    # Add additional features if available
    for col in ['Posterior Km', 'Anterior Km', 'Bio-ACD', 'Bio-LT']:
        if col in df_opt_train.columns:
            feature_cols.append(col)
    
    # Impute missing values
    imputer = KNNImputer(n_neighbors=5)
    X_train_imputed = imputer.fit_transform(df_opt_train[feature_cols])
    
    # Calculate residuals to predict
    df_opt_train['SRKT_ML_nc'] = df_opt_train.apply(
        lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Kerato'], 
                                  row['A-Constant'], nc=best_nc), 
        axis=1
    )
    df_opt_train['Expected_SE_ML_nc'] = -(df_opt_train['IOL Power'] - df_opt_train['SRKT_ML_nc'])
    df_opt_train['Error_ML_nc'] = df_opt_train['PostOP Spherical Equivalent'] - df_opt_train['Expected_SE_ML_nc']
    
    y_residuals = df_opt_train['Error_ML_nc'].values
    
    # Create ensemble with STRONGER regularization
    ensemble = VotingRegressor([
        ('ridge', Pipeline([
            ('scaler', RobustScaler()),
            ('ridge', Ridge(alpha=50.0))  # Increased from 10.0
        ])),
        ('huber', Pipeline([
            ('scaler', RobustScaler()),
            ('huber', HuberRegressor(epsilon=2.0, alpha=0.1))  # Increased regularization
        ])),
        ('rf', Pipeline([
            ('scaler', RobustScaler()),
            ('rf', RandomForestRegressor(n_estimators=10, max_depth=2,  # Simpler model
                                       min_samples_split=20, min_samples_leaf=15,
                                       random_state=SEED))
        ]))
    ])
    
    ensemble.fit(X_train_imputed, y_residuals)
    
    # TEST SET EVALUATION
    print("\nEvaluating on test set...")
    df_test_eval = df_test.copy()
    
    # Baseline
    df_test_eval['SRKT_Baseline'] = df_test_eval.apply(
        lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Kerato'], row['A-Constant']), 
        axis=1
    )
    df_test_eval['Expected_SE_Baseline'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_Baseline'])
    df_test_eval['Error_Baseline'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_Baseline']
    
    # ML-optimized nc
    df_test_eval['SRKT_ML_nc'] = df_test_eval.apply(
        lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Kerato'], 
                                  row['A-Constant'], nc=best_nc), 
        axis=1
    )
    df_test_eval['Expected_SE_ML_nc'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_ML_nc'])
    df_test_eval['Error_ML_nc'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_ML_nc']
    
    # Fully optimized
    df_test_eval['SRKT_ML_Full'] = df_test_eval.apply(
        lambda row: calculate_SRKT_ml(row['Bio-AL'], row['K_avg_Kerato'], 
                                     row['A-Constant'], optimal_params_all), 
        axis=1
    )
    df_test_eval['Expected_SE_ML_Full'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_ML_Full'])
    df_test_eval['Error_ML_Full'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_ML_Full']
    
    # Ensemble with conservative dampening (0.3 instead of 0.5)
    X_test_imputed = imputer.transform(df_test_eval[feature_cols])
    ensemble_corrections = ensemble.predict(X_test_imputed)
    df_test_eval['SRKT_Final'] = df_test_eval['SRKT_ML_nc'] + 0.3 * ensemble_corrections
    df_test_eval['Expected_SE_Final'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_Final'])
    df_test_eval['Error_Final'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_Final']
    
    # Calculate metrics
    mae_baseline = df_test_eval['Error_Baseline'].abs().mean()
    mae_nc = df_test_eval['Error_ML_nc'].abs().mean()
    mae_full = df_test_eval['Error_ML_Full'].abs().mean()
    mae_ensemble = df_test_eval['Error_Final'].abs().mean()
    
    # Store results
    seed_results.append({
        'seed': SEED,
        'train_size': len(df_train_val),
        'test_size': len(df_test),
        'optimal_nc': best_nc,
        'mae_baseline': mae_baseline,
        'mae_nc': mae_nc,
        'mae_full': mae_full,
        'mae_ensemble': mae_ensemble,
        'improvement_nc': (mae_baseline - mae_nc) / mae_baseline * 100,
        'improvement_full': (mae_baseline - mae_full) / mae_baseline * 100,
        'improvement_ensemble': (mae_baseline - mae_ensemble) / mae_baseline * 100,
        'optimal_params': optimal_params_all.tolist()
    })
    
    print(f"Test MAE - Baseline: {mae_baseline:.3f}, ML-nc: {mae_nc:.3f}, "
          f"ML-Full: {mae_full:.3f}, Ensemble: {mae_ensemble:.3f}")

# Convert to DataFrame
results_df = pd.DataFrame(seed_results)

# Save detailed results
results_df.to_csv('extended_bounds_results.csv', index=False)
print(f"\nDetailed results saved to: extended_bounds_results.csv")

In [ ]:
# %% Cell 4: Comprehensive Analysis of Extended Bounds Results
print("\n" + "="*80)
print("MULTI-SEED ANALYSIS RESULTS - EXTENDED BOUNDS")
print("="*80)

# Display all results
print("\nDetailed results for each seed:")
display_cols = ['seed', 'mae_baseline', 'mae_nc', 'mae_full', 'mae_ensemble', 
                'improvement_nc', 'improvement_full', 'improvement_ensemble']
print(results_df[display_cols].to_string(index=False, float_format='%.3f'))

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS ACROSS ALL SEEDS")
print("="*60)

summary_stats = results_df[['mae_baseline', 'mae_nc', 'mae_full', 'mae_ensemble', 
                            'improvement_nc', 'improvement_full', 'improvement_ensemble']].describe()
print(summary_stats)

# Key metrics
print("\n" + "="*60)
print("KEY PERFORMANCE METRICS (Mean ± Std)")
print("="*60)

metrics = {
    'Baseline SRK/T': results_df['mae_baseline'],
    'ML-Optimized nc': results_df['mae_nc'],
    'ML-Optimized Full': results_df['mae_full'],
    'ML + Ensemble': results_df['mae_ensemble']
}

for method, values in metrics.items():
    print(f"{method:20s}: MAE = {values.mean():.3f} ± {values.std():.3f} D "
          f"[{values.min():.3f}, {values.max():.3f}]")

print("\nIMPROVEMENT OVER BASELINE:")
improvements = {
    'NC optimization': results_df['improvement_nc'],
    'Full optimization': results_df['improvement_full'],
    'With ensemble': results_df['improvement_ensemble']
}

for method, values in improvements.items():
    print(f"{method:20s}: {values.mean():.1f}% ± {values.std():.1f}% "
          f"[{values.min():.1f}%, {values.max():.1f}%]")

# Check for overfitting signs
print("\n" + "="*60)
print("OVERFITTING CHECK")
print("="*60)

# Calculate coefficient of variation
for method, values in metrics.items():
    cv = values.std() / values.mean() * 100
    print(f"{method:20s}: CV = {cv:.1f}%")

# Statistical testing
print("\n" + "="*60)
print("STATISTICAL SIGNIFICANCE (Paired t-test across seeds)")
print("="*60)

from scipy.stats import ttest_rel

# Test each method against baseline
for method, col in [('ML-nc', 'mae_nc'), ('ML-Full', 'mae_full'), ('ML-Ensemble', 'mae_ensemble')]:
    t_stat, p_value = ttest_rel(results_df['mae_baseline'], results_df[col])
    mean_diff = results_df['mae_baseline'].mean() - results_df[col].mean()
    print(f"{method} vs Baseline: Mean diff = {mean_diff:.3f} D, "
          f"t = {t_stat:.3f}, p = {p_value:.4f}, "
          f"Significant: {'Yes' if p_value < 0.05 else 'No'}")

# Optimal parameter analysis
print("\n" + "="*60)
print("OPTIMAL PARAMETER RANGES")
print("="*60)

# Extract all optimal parameters
all_params = np.array([params for params in results_df['optimal_params']])
param_names = ['nc', 'cw_const1', 'cw_const2', 'cw_const3', 'acd_const1', 
               'acd_const2', 'offset_const', 'rethick_const1', 'rethick_const2']
standard_params = [1.333, -5.41, 0.58412, 0.098, 0.62467, 68.747, 3.336, 0.65696, 0.02029]

print(f"{'Parameter':15s} {'Standard':>10s} {'Mean Opt':>10s} {'Std':>8s} {'Min':>10s} {'Max':>10s} {'Change %':>10s}")
print("-" * 85)
for i, name in enumerate(param_names):
    param_values = all_params[:, i]
    mean_val = param_values.mean()
    std_val = param_values.std()
    min_val = param_values.min()
    max_val = param_values.max()
    change_pct = (mean_val - standard_params[i]) / abs(standard_params[i]) * 100
    
    print(f"{name:15s} {standard_params[i]:10.5f} {mean_val:10.5f} "
          f"{std_val:8.5f} {min_val:10.5f} {max_val:10.5f} {change_pct:10.2f}%")

# Warning about parameter ranges
if any(abs((all_params[:, i].mean() - standard_params[i]) / standard_params[i]) > 0.1 
       for i in range(len(param_names))):
    print("\nWARNING: Some parameters deviate >10% from standard values.")
    print("This may indicate overfitting. Consider clinical validation.")

In [ ]:
# %% Cell 5: Comprehensive Visualization with Extended Bounds Analysis
fig = plt.figure(figsize=(24, 18))

# 1. MAE comparison across seeds
ax1 = plt.subplot(4, 3, 1)
x = np.arange(len(SEEDS))
width = 0.2
ax1.bar(x - 1.5*width, results_df['mae_baseline'], width, label='Baseline', color='lightblue')
ax1.bar(x - 0.5*width, results_df['mae_nc'], width, label='ML-nc', color='lightgreen')
ax1.bar(x + 0.5*width, results_df['mae_full'], width, label='ML-Full', color='lightcoral')
ax1.bar(x + 1.5*width, results_df['mae_ensemble'], width, label='Ensemble', color='gold')
ax1.set_xlabel('Seed')
ax1.set_ylabel('MAE (D)')
ax1.set_title('MAE Across Different Seeds (Extended Bounds)')
ax1.set_xticks(x)
ax1.set_xticklabels(SEEDS, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Box plot of MAE distributions
ax2 = plt.subplot(4, 3, 2)
data_to_plot = [results_df['mae_baseline'], results_df['mae_nc'], 
                results_df['mae_full'], results_df['mae_ensemble']]
bp = ax2.boxplot(data_to_plot, labels=['Baseline', 'ML-nc', 'ML-Full', 'Ensemble'],
                 patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightcoral', 'gold']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
ax2.set_ylabel('MAE (D)')
ax2.set_title('MAE Distribution - Extended Bounds')
ax2.grid(True, alpha=0.3)

# 3. Improvement percentages
ax3 = plt.subplot(4, 3, 3)
improvement_data = [results_df['improvement_nc'], results_df['improvement_full'], 
                   results_df['improvement_ensemble']]
bp2 = ax3.boxplot(improvement_data, labels=['NC Only', 'Full Opt', 'Ensemble'],
                  patch_artist=True)
for patch, color in zip(bp2['boxes'], ['lightgreen', 'lightcoral', 'gold']):
    patch.set_facecolor(color)
ax3.set_ylabel('Improvement (%)')
ax3.set_title('Improvement Distribution')
ax3.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax3.axhline(y=30, color='green', linestyle='--', alpha=0.5, label='30% target')
ax3.grid(True, alpha=0.3)
ax3.legend()

# 4. Parameter deviation heatmap
ax4 = plt.subplot(4, 3, 4)
all_params = np.array([params for params in results_df['optimal_params']])
param_deviations = np.zeros((len(SEEDS), len(param_names)))
for i in range(len(param_names)):
    param_deviations[:, i] = (all_params[:, i] - standard_params[i]) / abs(standard_params[i]) * 100

im = ax4.imshow(param_deviations.T, cmap='RdBu_r', aspect='auto', vmin=-30, vmax=30)
ax4.set_yticks(range(len(param_names)))
ax4.set_yticklabels(param_names)
ax4.set_xticks(range(len(SEEDS)))
ax4.set_xticklabels(SEEDS, rotation=45)
ax4.set_xlabel('Seed')
ax4.set_title('Parameter Deviation from Standard (%)')
plt.colorbar(im, ax=ax4, label='Deviation %')

# 5. NC values across seeds with extended range
ax5 = plt.subplot(4, 3, 5)
ax5.scatter(SEEDS, results_df['optimal_nc'], s=100, c='blue', alpha=0.7)
ax5.axhline(y=1.333, color='red', linestyle='--', label='Standard nc=1.333')
ax5.axhline(y=results_df['optimal_nc'].mean(), color='green', 
            linestyle='--', label=f'Mean nc={results_df["optimal_nc"].mean():.5f}')
ax5.fill_between(SEEDS, 1.325, 1.345, alpha=0.2, color='gray', label='Extended bounds')
ax5.set_xlabel('Seed')
ax5.set_ylabel('Optimal nc')
ax5.set_title('Optimal nc Values (Extended Range)')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Best vs worst seed comparison
ax6 = plt.subplot(4, 3, 6)
best_seed_idx = results_df['improvement_full'].idxmax()
worst_seed_idx = results_df['improvement_full'].idxmin()
methods = ['Baseline', 'NC', 'Full', 'Ensemble']
best_maes = [results_df.loc[best_seed_idx, 'mae_baseline'],
             results_df.loc[best_seed_idx, 'mae_nc'],
             results_df.loc[best_seed_idx, 'mae_full'],
             results_df.loc[best_seed_idx, 'mae_ensemble']]
worst_maes = [results_df.loc[worst_seed_idx, 'mae_baseline'],
              results_df.loc[worst_seed_idx, 'mae_nc'],
              results_df.loc[worst_seed_idx, 'mae_full'],
              results_df.loc[worst_seed_idx, 'mae_ensemble']]

x_pos = np.arange(len(methods))
ax6.bar(x_pos - 0.2, best_maes, 0.4, label=f'Best (Seed {results_df.loc[best_seed_idx, "seed"]})', 
        color='green', alpha=0.7)
ax6.bar(x_pos + 0.2, worst_maes, 0.4, label=f'Worst (Seed {results_df.loc[worst_seed_idx, "seed"]})', 
        color='red', alpha=0.7)
ax6.set_xticks(x_pos)
ax6.set_xticklabels(methods)
ax6.set_ylabel('MAE (D)')
ax6.set_title('Best vs Worst Seed Performance')
ax6.legend()
ax6.grid(True, alpha=0.3, axis='y')

# 7. Stability analysis
ax7 = plt.subplot(4, 3, 7)
stability_metrics = []
for col in ['mae_baseline', 'mae_nc', 'mae_full', 'mae_ensemble']:
    cv = results_df[col].std() / results_df[col].mean() * 100
    stability_metrics.append(cv)

bars = ax7.bar(methods, stability_metrics, color=colors)
ax7.set_ylabel('Coefficient of Variation (%)')
ax7.set_title('Method Stability (Lower is Better)')
ax7.axhline(y=10, color='green', linestyle='--', alpha=0.5, label='Good stability')
ax7.axhline(y=20, color='orange', linestyle='--', alpha=0.5, label='Moderate stability')
ax7.axhline(y=30, color='red', linestyle='--', alpha=0.5, label='Poor stability')
ax7.legend()
ax7.grid(True, alpha=0.3, axis='y')

# 8. Improvement vs parameter deviation
ax8 = plt.subplot(4, 3, 8)
max_param_deviation = np.max(np.abs(param_deviations), axis=1)
ax8.scatter(max_param_deviation, results_df['improvement_full'], s=100, alpha=0.7)
ax8.set_xlabel('Max Parameter Deviation (%)')
ax8.set_ylabel('Improvement (%)')
ax8.set_title('Improvement vs Parameter Deviation')
ax8.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(max_param_deviation, results_df['improvement_full'], 1)
p = np.poly1d(z)
ax8.plot(sorted(max_param_deviation), p(sorted(max_param_deviation)), "r--", alpha=0.8)

# 9. Cumulative improvement distribution
ax9 = plt.subplot(4, 3, 9)
for method, col, color in [('NC', 'improvement_nc', 'green'),
                          ('Full', 'improvement_full', 'red'),
                          ('Ensemble', 'improvement_ensemble', 'gold')]:
    sorted_imp = np.sort(results_df[col])
    cumulative = np.arange(1, len(sorted_imp) + 1) / len(sorted_imp)
    ax9.plot(sorted_imp, cumulative, label=method, color=color, linewidth=2)
ax9.axvline(x=30, color='black', linestyle='--', alpha=0.5, label='30% target')
ax9.set_xlabel('Improvement (%)')
ax9.set_ylabel('Cumulative Probability')
ax9.set_title('Probability of Achieving Improvement')
ax9.legend()
ax9.grid(True, alpha=0.3)

# 10. Parameter correlation matrix
ax10 = plt.subplot(4, 3, 10)
param_corr = np.corrcoef(all_params.T)
im = ax10.imshow(param_corr, cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
ax10.set_xticks(range(len(param_names)))
ax10.set_yticks(range(len(param_names)))
ax10.set_xticklabels(param_names, rotation=45)
ax10.set_yticklabels(param_names)
ax10.set_title('Parameter Correlation Matrix')
plt.colorbar(im, ax=ax10)

# 11. Summary statistics
ax11 = plt.subplot(4, 3, 11)
ax11.axis('off')
summary_text = f"""
EXTENDED BOUNDS ANALYSIS SUMMARY (n={len(SEEDS)} seeds)

Best Method: ML-Full Optimization
MAE: {results_df['mae_full'].mean():.3f} ± {results_df['mae_full'].std():.3f} D
Improvement: {results_df['improvement_full'].mean():.1f}% ± {results_df['improvement_full'].std():.1f}%
Range: [{results_df['improvement_full'].min():.1f}%, {results_df['improvement_full'].max():.1f}%]

Optimal nc: {results_df['optimal_nc'].mean():.5f} ± {results_df['optimal_nc'].std():.5f}
Range tested: [1.325, 1.345] (extended from [1.330, 1.338])

Success rate:
- Improvement > 25%: {(results_df['improvement_full'] > 25).sum()}/{len(SEEDS)} seeds
- Improvement > 30%: {(results_df['improvement_full'] > 30).sum()}/{len(SEEDS)} seeds
- Improvement > 35%: {(results_df['improvement_full'] > 35).sum()}/{len(SEEDS)} seeds

Warning: Extended bounds may increase overfitting risk
Recommend clinical validation before implementation
"""
ax11.text(0.05, 0.5, summary_text, fontsize=11, verticalalignment='center',
         fontfamily='monospace', bbox=dict(boxstyle="round,pad=0.5", 
         facecolor="lightyellow", alpha=0.8))

# 12. Risk assessment
ax12 = plt.subplot(4, 3, 12)
risk_scores = []
for idx in range(len(results_df)):
    # Risk based on parameter deviation and stability
    param_dev = np.max(np.abs(param_deviations[idx]))
    improvement = results_df.loc[idx, 'improvement_full']
    # High risk if large deviation with small improvement
    risk = param_dev / (improvement + 10)  # Add 10 to avoid division issues
    risk_scores.append(risk)

ax12.scatter(SEEDS, risk_scores, s=100, c=risk_scores, cmap='RdYlGn_r')
ax12.set_xlabel('Seed')
ax12.set_ylabel('Risk Score')
ax12.set_title('Overfitting Risk Assessment (Lower is Better)')
ax12.grid(True, alpha=0.3)

plt.suptitle('Extended Bounds Multi-Seed Analysis for FacoDMEK IOL Optimization', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# %% Cell 6: Clinical Recommendations with Extended Bounds Risk Assessment
print("\n" + "="*80)
print("CLINICAL RECOMMENDATIONS - EXTENDED BOUNDS ANALYSIS")
print("="*80)

# Calculate robust estimates
mean_baseline = results_df['mae_baseline'].mean()
mean_optimized = results_df['mae_full'].mean()
mean_improvement = results_df['improvement_full'].mean()
std_improvement = results_df['improvement_full'].std()

# Parameter analysis
all_params = np.array([params for params in results_df['optimal_params']])
param_names = ['nc', 'cw_const1', 'cw_const2', 'cw_const3', 'acd_const1', 
               'acd_const2', 'offset_const', 'rethick_const1', 'rethick_const2']
standard_params = [1.333, -5.41, 0.58412, 0.098, 0.62467, 68.747, 3.336, 0.65696, 0.02029]

print(f"\n1. PERFORMANCE SUMMARY:")
print(f"   Standard SRK/T: MAE = {mean_baseline:.3f} D")
print(f"   Optimized SRK/T: MAE = {mean_optimized:.3f} D")
print(f"   Mean improvement: {mean_improvement:.1f}% ± {std_improvement:.1f}%")
print(f"   Best case: {results_df['improvement_full'].max():.1f}%")
print(f"   Worst case: {results_df['improvement_full'].min():.1f}%")

print(f"\n2. PARAMETER RECOMMENDATIONS:")
print(f"   {'Parameter':15s} {'Standard':>10s} {'Recommended':>12s} {'Range':>20s}")
print("   " + "-"*60)
for i, name in enumerate(param_names):
    param_values = all_params[:, i]
    mean_val = param_values.mean()
    std_val = param_values.std()
    
    # Flag high-risk parameters (>10% deviation)
    deviation = abs((mean_val - standard_params[i]) / standard_params[i] * 100)
    risk_flag = " ⚠️" if deviation > 10 else ""
    
    print(f"   {name:15s} {standard_params[i]:10.5f} {mean_val:12.5f} "
          f"[{mean_val-std_val:.5f}, {mean_val+std_val:.5f}]{risk_flag}")

print(f"\n3. RISK ASSESSMENT:")
max_deviations = [np.max(np.abs((all_params[i] - standard_params) / standard_params * 100)) 
                  for i in range(len(all_params))]
avg_max_deviation = np.mean(max_deviations)
print(f"   Average maximum parameter deviation: {avg_max_deviation:.1f}%")
print(f"   Seeds with >20% deviation: {sum(d > 20 for d in max_deviations)}/{len(SEEDS)}")
print(f"   Risk level: {'HIGH' if avg_max_deviation > 20 else 'MODERATE' if avg_max_deviation > 10 else 'LOW'}")

print(f"\n4. IMPLEMENTATION STRATEGY:")
print(f"   a) Conservative: Use nc = {results_df['optimal_nc'].mean():.5f} only")
print(f"      Expected improvement: {results_df['improvement_nc'].mean():.1f}%")
print(f"   b) Moderate: Use optimized parameters with <10% deviation")
print(f"   c) Aggressive: Use full optimization (with clinical validation)")

print(f"\n5. VALIDATION REQUIREMENTS:")
print(f"   ⚠️  Extended bounds require additional validation:")
print(f"   - Test on independent dataset")
print(f"   - Monitor first 50-100 cases closely")
print(f"   - Compare with standard formula outcomes")
print(f"   - Consider gradual implementation")

# Statistical significance check
from scipy.stats import ttest_rel
t_stat, p_value = ttest_rel(results_df['mae_baseline'], results_df['mae_full'])
print(f"\n6. STATISTICAL SIGNIFICANCE:")
print(f"   Paired t-test p-value: {p_value:.4f}")
print(f"   Significant at α=0.05: {'Yes ✓' if p_value < 0.05 else 'No ✗'}")
print(f"   Effect size (Cohen's d): {(mean_baseline - mean_optimized) / np.sqrt((results_df['mae_baseline'].std()**2 + results_df['mae_full'].std()**2) / 2):.3f}")

# Export recommendations
recommendations = {
    'analysis_type': 'Extended Bounds Multi-Seed',
    'n_seeds': len(SEEDS),
    'n_eyes': len(df_complete),
    'performance': {
        'baseline_mae': float(mean_baseline),
        'optimized_mae': float(mean_optimized),
        'mean_improvement_pct': float(mean_improvement),
        'std_improvement_pct': float(std_improvement),
        'p_value': float(p_value)
    },
    'parameters': {
        name: {
            'standard': float(standard_params[i]),
            'optimized_mean': float(all_params[:, i].mean()),
            'optimized_std': float(all_params[:, i].std()),
            'deviation_pct': float((all_params[:, i].mean() - standard_params[i]) / abs(standard_params[i]) * 100)
        }
        for i, name in enumerate(param_names)
    }
}

import json
with open('extended_bounds_recommendations.json', 'w') as f:
    json.dump(recommendations, f, indent=2)

print(f"\n7. FILES SAVED:")
print(f"   - extended_bounds_results.csv")
print(f"   - extended_bounds_recommendations.json")

print("\n" + "="*80)

In [ ]:
# %% Cell 6.5: Optimize ALL SRK/T Formula Constants
print("\nOPTIMIZING ALL SRK/T FORMULA CONSTANTS")
print("="*60)
print("This will optimize all 9 parameters, not just nc")
print(f"Using random seed: {RANDOM_SEED}")
print("="*60)

# Extended SRK/T with learnable parameters
def calculate_SRKT_ml(AL, K, A_const, params):
    """
    ML-optimized SRK/T formula with all parameters optimizable
    params = [nc, cw_const1, cw_const2, cw_const3, acd_const1, acd_const2, 
              offset_const, rethick_const1, rethick_const2]
    """
    # Unpack all parameters
    nc = params[0]
    cw_const1, cw_const2, cw_const3 = params[1:4]
    acd_const1, acd_const2 = params[4:6]
    offset_const = params[6]
    rethick_const1, rethick_const2 = params[7:9]
    
    if pd.isna(AL) or pd.isna(K) or pd.isna(A_const) or K <= 0 or AL <= 0:
        return np.nan
    
    try:
        na = 1.336
        r = 337.5 / K
        
        # Use standard threshold
        if AL > 24.2:
            LCOR = -3.446 + 1.716 * AL - 0.0237 * AL**2
        else:
            LCOR = AL
        
        # Optimized corneal width calculation
        Cw = cw_const1 + cw_const2 * LCOR + cw_const3 * K
        
        if r**2 - (Cw**2 / 4) < 0:
            return np.nan
        
        H = r - np.sqrt(r**2 - (Cw**2 / 4))
        
        # Optimized ACD calculation
        ACDconst = acd_const1 * A_const - acd_const2
        offset = ACDconst - offset_const
        ACDest = H + offset
        
        # Optimized retinal thickness
        RETHICK = rethick_const1 - rethick_const2 * AL
        LOPT = AL + RETHICK
        
        ncm1 = nc - 1
        IOL = (1000 * na * (na * r - ncm1 * LOPT)) / ((LOPT - ACDest) * (na * r - ncm1 * ACDest))
        
        return IOL
    except:
        return np.nan

# Objective function for optimizing ALL constants
def objective_all_constants(params):
    """Cross-validation based objective with strong regularization"""
    errors_all = []
    
    # 3-fold cross-validation
    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)  # USE VARIABLE SEED
    
    for train_idx, val_idx in kf.split(df_opt_train):
        val_data = df_opt_train.iloc[val_idx]
        
        predictions = val_data.apply(
            lambda row: calculate_SRKT_ml(row['Bio-AL'], row['K_avg_Kerato'], 
                                         row['A-Constant'], params), 
            axis=1
        )
        
        expected_se = -(val_data['IOL Power'] - predictions)
        errors = val_data['PostOP Spherical Equivalent'] - expected_se
        
        valid_errors = errors[errors.notna()]
        if len(valid_errors) > 0:
            errors_all.extend(valid_errors.abs().tolist())
    
    if len(errors_all) == 0:
        return 999
    
    mae = np.mean(errors_all)
    
    # Add regularization to prevent extreme values
    # Penalize deviation from standard values
    standard_params = [1.333, -5.41, 0.58412, 0.098, 0.62467, 68.747, 3.336, 0.65696, 0.02029]
    penalty = 0
    for i, (opt, std) in enumerate(zip(params, standard_params)):
        # Different weights for different parameters
        if i == 0:  # nc - allow more flexibility
            penalty += 5 * ((opt - std) / std) ** 2
        else:  # other parameters - stronger regularization
            penalty += 20 * ((opt - std) / std) ** 2
    
    return mae + penalty

# Initial parameters (standard SRK/T values)
initial_params = [
    1.333,          # nc
    -5.41,          # cw_const1
    0.58412,        # cw_const2
    0.098,          # cw_const3
    0.62467,        # acd_const1
    68.747,         # acd_const2
    3.336,          # offset_const
    0.65696,        # rethick_const1
    0.02029         # rethick_const2
]

# Bounds for optimization (tighter bounds for stability)
bounds = [
    (1.330, 1.340),           # nc
    (-5.8, -5.0),             # cw_const1
    (0.55, 0.62),             # cw_const2
    (0.090, 0.106),           # cw_const3
    (0.60, 0.65),             # acd_const1
    (65.0, 72.0),             # acd_const2
    (3.1, 3.6),               # offset_const
    (0.63, 0.68),             # rethick_const1
    (0.018, 0.023)            # rethick_const2
]

print("\nOptimizing SRK/T formula constants using Differential Evolution...")
print("This may take a few minutes...")

# Run optimization with more conservative settings
result_de = differential_evolution(
    objective_all_constants, 
    bounds,
    maxiter=50,     # More iterations
    popsize=15,
    seed=RANDOM_SEED,  # USE VARIABLE SEED
    atol=0.0001,
    tol=0.0001,
    disp=True,
    workers=1,
    strategy='best1bin',
    mutation=(0.5, 1),  # Conservative mutation
    recombination=0.7
)

optimal_params_all = result_de.x

print(f"\nOptimization complete!")
print(f"Final objective value: {result_de.fun:.4f}")

# Display optimized parameters
param_names = ['nc', 'cw_const1', 'cw_const2', 'cw_const3', 'acd_const1', 
               'acd_const2', 'offset_const', 'rethick_const1', 'rethick_const2']

print("\nOptimized Parameters:")
print("-" * 60)
for name, original, optimized in zip(param_names, initial_params, optimal_params_all):
    change_pct = (optimized - original) / abs(original) * 100
    print(f"{name:20s}: {original:8.5f} → {optimized:8.5f} ({change_pct:+6.2f}%)")

# Calculate performance with fully optimized formula
df_opt_train['SRKT_ML_Full'] = df_opt_train.apply(
    lambda row: calculate_SRKT_ml(row['Bio-AL'], row['K_avg_Kerato'], 
                                 row['A-Constant'], optimal_params_all), 
    axis=1
)

df_opt_train['Expected_SE_ML_Full'] = -(df_opt_train['IOL Power'] - df_opt_train['SRKT_ML_Full'])
df_opt_train['Error_ML_Full'] = df_opt_train['PostOP Spherical Equivalent'] - df_opt_train['Expected_SE_ML_Full']

# Compare performances
print("\n" + "="*60)
print("TRAINING SET PERFORMANCE COMPARISON:")
print("="*60)

baseline_train_mae = df_opt_train['SRKT_Error'].abs().mean()
nc_only_mae = df_opt_train['Error_ML_nc'].abs().mean()
full_opt_mae = df_opt_train['Error_ML_Full'].abs().mean()

print(f"Baseline SRK/T:           MAE = {baseline_train_mae:.3f} D")
print(f"Optimized nc only:        MAE = {nc_only_mae:.3f} D ({(baseline_train_mae-nc_only_mae)/baseline_train_mae*100:.1f}% improvement)")
print(f"Fully optimized SRK/T:    MAE = {full_opt_mae:.3f} D ({(baseline_train_mae-full_opt_mae)/baseline_train_mae*100:.1f}% improvement)")

# Update model_info to include full optimization
model_info['optimal_params_all'] = optimal_params_all
model_info['param_names'] = param_names
model_info['calculate_SRKT_ml'] = calculate_SRKT_ml

print("\nFull formula optimization complete!")

In [ ]:
# %% Cell 7: Final Test Set Evaluation with Bootstrap Confidence Intervals
print("\n" + "#"*80)
print("#" + " "*30 + "FINAL TEST SET EVALUATION" + " "*24 + "#")
print("#"*80)
print("\nUsing bootstrap for more reliable significance testing")
print(f"Random seed: {RANDOM_SEED}")
print("="*80)

# Prepare test data
df_test_eval = df_test.copy()

# 1. Baseline SRK/T
print("\nCalculating baseline SRK/T...")
df_test_eval['SRKT_Baseline'] = df_test_eval.apply(
    lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Kerato'], 
                              row['A-Constant'], nc=1.333), 
    axis=1
)
df_test_eval['Expected_SE_Baseline'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_Baseline'])
df_test_eval['Error_Baseline'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_Baseline']

# 2. ML-optimized nc only
print("Calculating ML-optimized nc...")
df_test_eval['SRKT_ML_nc'] = df_test_eval.apply(
    lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Kerato'], 
                              row['A-Constant'], nc=model_info['optimal_nc']), 
    axis=1
)
df_test_eval['Expected_SE_ML_nc'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_ML_nc'])
df_test_eval['Error_ML_nc'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_ML_nc']

# 3. Fully optimized SRK/T (all constants)
print("Calculating fully optimized SRK/T...")
if 'optimal_params_all' in model_info:
    df_test_eval['SRKT_ML_Full'] = df_test_eval.apply(
        lambda row: calculate_SRKT_ml(row['Bio-AL'], row['K_avg_Kerato'], 
                                     row['A-Constant'], model_info['optimal_params_all']), 
        axis=1
    )
    df_test_eval['Expected_SE_ML_Full'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_ML_Full'])
    df_test_eval['Error_ML_Full'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_ML_Full']

# 4. ML + Ensemble
print("Calculating ensemble predictions...")
# Prepare features using the same imputer from training
X_test = df_test_eval[model_info['feature_cols']].copy()

# Use the fitted imputer from training
X_test_imputed = pd.DataFrame(
    model_info['imputer'].transform(X_test),
    columns=model_info['feature_cols'],
    index=X_test.index
)

# Make ensemble predictions
ensemble_corrections = model_info['ensemble'].predict(X_test_imputed)
df_test_eval['SRKT_Final'] = (
    df_test_eval['SRKT_ML_nc'] + 
    model_info['dampening_factor'] * ensemble_corrections
)
df_test_eval['Expected_SE_Final'] = -(df_test_eval['IOL Power'] - df_test_eval['SRKT_Final'])
df_test_eval['Error_Final'] = df_test_eval['PostOP Spherical Equivalent'] - df_test_eval['Expected_SE_Final']

# Calculate metrics
print("\n" + "="*60)
print("TEST SET RESULTS:")
print("="*60)

methods = {
    'Baseline SRK/T': df_test_eval['Error_Baseline'].abs(),
    'ML-Optimized nc': df_test_eval['Error_ML_nc'].abs(),
}

# Add fully optimized if available
if 'Error_ML_Full' in df_test_eval.columns:
    methods['ML-Optimized Full'] = df_test_eval['Error_ML_Full'].abs()

methods['ML + Ensemble'] = df_test_eval['Error_Final'].abs()

# Results summary
results_data = []
for method, errors in methods.items():
    valid_errors = errors.dropna()
    mae = valid_errors.mean()
    std = valid_errors.std()
    within_025 = (valid_errors <= 0.25).sum() / len(valid_errors) * 100
    within_050 = (valid_errors <= 0.50).sum() / len(valid_errors) * 100
    within_100 = (valid_errors <= 1.00).sum() / len(valid_errors) * 100
    
    results_data.append({
        'Method': method,
        'MAE (D)': mae,
        'SD (D)': std,
        '±0.25D (%)': within_025,
        '±0.50D (%)': within_050,
        '±1.00D (%)': within_100
    })
    
results_df = pd.DataFrame(results_data)
print(results_df.to_string(index=False, float_format=lambda x: f'{x:.3f}' if x < 10 else f'{x:.1f}'))

# Bootstrap significance testing with CORRECTED function
print("\n" + "="*60)
print("BOOTSTRAP SIGNIFICANCE TESTING (1000 iterations)")
print("="*60)

def bootstrap_paired_test(errors1, errors2, n_bootstrap=1000):
    """Bootstrap test for paired samples - RETURNS bootstrap_diffs"""
    # Align indices
    common_idx = errors1.index.intersection(errors2.index)
    e1 = errors1[common_idx].values
    e2 = errors2[common_idx].values
    
    # Calculate observed difference
    observed_diff = np.mean(e1) - np.mean(e2)
    
    # Bootstrap
    bootstrap_diffs = []
    n_samples = len(e1)
    
    # Set seed for bootstrap
    np.random.seed(RANDOM_SEED)
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        idx = np.random.choice(n_samples, n_samples, replace=True)
        boot_diff = np.mean(e1[idx]) - np.mean(e2[idx])
        bootstrap_diffs.append(boot_diff)
    
    bootstrap_diffs = np.array(bootstrap_diffs)
    
    # Calculate p-value (two-sided)
    p_value = np.sum(bootstrap_diffs <= 0) / n_bootstrap * 2
    p_value = min(p_value, 1.0)
    
    # Confidence interval
    ci_lower = np.percentile(bootstrap_diffs, 2.5)
    ci_upper = np.percentile(bootstrap_diffs, 97.5)
    
    return observed_diff, p_value, ci_lower, ci_upper, bootstrap_diffs

# Test ML methods against baseline
baseline_errors = df_test_eval['Error_Baseline'].abs().dropna()

# 1. ML-nc vs Baseline
print("\n1. ML-Optimized nc vs Baseline:")
ml_nc_errors = df_test_eval['Error_ML_nc'].abs().dropna()
diff_nc, p_nc, ci_lower_nc, ci_upper_nc, bootstrap_diffs_nc = bootstrap_paired_test(
    baseline_errors, ml_nc_errors
)
print(f"   Mean difference: {diff_nc:.4f} D")
print(f"   95% CI: [{ci_lower_nc:.4f}, {ci_upper_nc:.4f}]")
print(f"   Bootstrap p-value: {p_nc:.4f}")
print(f"   Significant at α=0.05: {'Yes' if p_nc < 0.05 else 'No'}")

# 2. Fully optimized vs Baseline (if available)
if 'Error_ML_Full' in df_test_eval.columns:
    print("\n2. Fully Optimized SRK/T vs Baseline:")
    ml_full_errors = df_test_eval['Error_ML_Full'].abs().dropna()
    diff_full, p_full, ci_lower_full, ci_upper_full, bootstrap_diffs_full = bootstrap_paired_test(
        baseline_errors, ml_full_errors
    )
    print(f"   Mean difference: {diff_full:.4f} D")
    print(f"   95% CI: [{ci_lower_full:.4f}, {ci_upper_full:.4f}]")
    print(f"   Bootstrap p-value: {p_full:.4f}")
    print(f"   Significant at α=0.05: {'Yes' if p_full < 0.05 else 'No'}")

# 3. ML+Ensemble vs Baseline
print("\n3. ML + Ensemble vs Baseline:")
final_errors = df_test_eval['Error_Final'].abs().dropna()
diff_final, p_final, ci_lower_final, ci_upper_final, bootstrap_diffs = bootstrap_paired_test(
    baseline_errors, final_errors
)
print(f"   Mean difference: {diff_final:.4f} D")
print(f"   95% CI: [{ci_lower_final:.4f}, {ci_upper_final:.4f}]")
print(f"   Bootstrap p-value: {p_final:.4f}")
print(f"   Significant at α=0.05: {'Yes' if p_final < 0.05 else 'No'}")

# Additional paired t-test for comparison
from scipy.stats import ttest_rel, wilcoxon

print("\n" + "="*60)
print("ADDITIONAL STATISTICAL TESTS")
print("="*60)

# Paired t-test
common_idx = baseline_errors.index.intersection(final_errors.index)
if len(common_idx) > 0:
    t_stat, p_ttest = ttest_rel(baseline_errors[common_idx], final_errors[common_idx])
    print(f"\nPaired t-test (ML + Ensemble vs Baseline):")
    print(f"   p-value: {p_ttest:.4f}")
    print(f"   Significant at α=0.05: {'Yes' if p_ttest < 0.05 else 'No'}")
    
    # Wilcoxon signed-rank test
    w_stat, p_wilcoxon = wilcoxon(baseline_errors[common_idx], final_errors[common_idx])
    print(f"\nWilcoxon signed-rank test (ML + Ensemble vs Baseline):")
    print(f"   p-value: {p_wilcoxon:.4f}")
    print(f"   Significant at α=0.05: {'Yes' if p_wilcoxon < 0.05 else 'No'}")

# Visualization
fig = plt.figure(figsize=(18, 10))

# Plot 1: Error distributions
ax1 = plt.subplot(2, 3, 1)
data_to_plot = []
labels = []
for method, errors in methods.items():
    if not errors.isna().all():
        data_to_plot.append(errors.dropna())
        labels.append(method)

bp = ax1.boxplot(data_to_plot, labels=labels, patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightcoral', 'gold']
for patch, color in zip(bp['boxes'], colors[:len(bp['boxes'])]):
    patch.set_facecolor(color)
ax1.set_ylabel('Absolute Error (D)')
ax1.set_title('Test Set Error Distributions')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Plot 2: Bootstrap distribution (final model)
ax2 = plt.subplot(2, 3, 2)
ax2.hist(bootstrap_diffs, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(x=0, color='red', linestyle='--', label='No difference')
ax2.axvline(x=diff_final, color='green', linestyle='-', linewidth=2, 
            label=f'Observed diff: {diff_final:.3f}')
ax2.set_xlabel('MAE Difference (Baseline - ML+Ensemble)')
ax2.set_ylabel('Frequency')
ax2.set_title('Bootstrap Distribution')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Performance comparison bar chart
ax3 = plt.subplot(2, 3, 3)
x_pos = np.arange(len(results_df))
ax3.bar(x_pos, results_df['MAE (D)'], color=colors[:len(results_df)])
ax3.set_xticks(x_pos)
ax3.set_xticklabels(results_df['Method'], rotation=45, ha='right')
ax3.set_ylabel('MAE (D)')
ax3.set_title('Mean Absolute Error Comparison')
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Percentage within target
ax4 = plt.subplot(2, 3, 4)
width = 0.25
x_pos = np.arange(len(results_df))
ax4.bar(x_pos - width, results_df['±0.25D (%)'], width, label='±0.25D', color='darkgreen')
ax4.bar(x_pos, results_df['±0.50D (%)'], width, label='±0.50D', color='orange')
ax4.bar(x_pos + width, results_df['±1.00D (%)'], width, label='±1.00D', color='blue')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(results_df['Method'], rotation=45, ha='right')
ax4.set_ylabel('Percentage (%)')
ax4.set_title('Percentage Within Target Range')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

# Plot 5: Scatter plot of best method
ax5 = plt.subplot(2, 3, 5)
valid_final = df_test_eval['Expected_SE_Final'].notna()
if valid_final.sum() > 0:
    ax5.scatter(df_test_eval.loc[valid_final, 'Expected_SE_Final'], 
               df_test_eval.loc[valid_final, 'PostOP Spherical Equivalent'], 
               alpha=0.6, color='gold')
    
    # Add perfect prediction line
    min_val = df_test_eval.loc[valid_final, 'Expected_SE_Final'].min()
    max_val = df_test_eval.loc[valid_final, 'Expected_SE_Final'].max()
    ax5.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')
    ax5.set_xlabel('Predicted SE (D)')
    ax5.set_ylabel('Actual SE (D)')
    ax5.set_title('ML + Ensemble: Predicted vs Actual')
    ax5.legend()
    ax5.grid(True, alpha=0.3)

# Plot 6: Improvement summary
ax6 = plt.subplot(2, 3, 6)
baseline_mae = results_df.loc[results_df['Method'] == 'Baseline SRK/T', 'MAE (D)'].values[0]
improvements = []
for _, row in results_df.iterrows():
    improvement = (baseline_mae - row['MAE (D)']) / baseline_mae * 100
    improvements.append(improvement)

bars = ax6.bar(range(len(results_df)), improvements, color=colors[:len(results_df)])
ax6.set_ylabel('Improvement over Baseline (%)')
ax6.set_title('Relative Performance Improvement')
ax6.set_xticks(range(len(results_df)))
ax6.set_xticklabels(results_df['Method'], rotation=45, ha='right')
ax6.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax6.grid(True, alpha=0.3)

# Add value labels on bars
for bar, imp in zip(bars, improvements):
    height = bar.get_height()
    ax6.annotate(f'{imp:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3 if height > 0 else -15),
                textcoords="offset points",
                ha='center', va='bottom' if height > 0 else 'top')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("\nKey findings:")
print(f"1. Test set size: {len(df_test)} eyes (30% of total)")
print(f"2. Best performing method: {results_df.loc[results_df['MAE (D)'].idxmin(), 'Method']}")
print(f"3. Best MAE: {results_df['MAE (D)'].min():.3f} D")
print(f"4. Improvement over baseline: {improvements[results_df['MAE (D)'].idxmin()]:.1f}%")
print(f"5. Random seed used: {RANDOM_SEED}")
print("="*80)

In [ ]:
# %% Cell 8: Document the Optimized SRK/T Formula for FacoDMEK Eyes
print("=" * 80)
print("OPTIMIZED SRK/T FORMULA FOR FacoDMEK EYES")
print("=" * 80)
print(f"\nBased on machine learning optimization of {len(df_complete)} FacoDMEK cases")
print("=" * 80)

# Section 1: NC-ONLY OPTIMIZATION
print("\n1. NC-ONLY OPTIMIZATION:")
print("-" * 60)
print(f"Original corneal refractive index (nc): 1.3330")
print(f"Optimized corneal refractive index (nc): {model_info['optimal_nc']:.5f}")
print(f"Change: {(model_info['optimal_nc'] - 1.333):.5f} ({((model_info['optimal_nc'] - 1.333)/1.333 * 100):.2f}%)")

# Section 2: FULL PARAMETER OPTIMIZATION
if 'optimal_params_all' in model_info:
    print("\n2. FULL PARAMETER OPTIMIZATION:")
    print("-" * 60)
    print("All optimized SRK/T parameters:")
    print()
    
    standard_params = [1.333, -5.41, 0.58412, 0.098, 0.62467, 68.747, 3.336, 0.65696, 0.02029]
    param_names = ['nc', 'cw_const1', 'cw_const2', 'cw_const3', 'acd_const1', 
                   'acd_const2', 'offset_const', 'rethick_const1', 'rethick_const2']
    
    # Create comparison table
    comparison_data = []
    for i, name in enumerate(param_names):
        original = standard_params[i]
        optimized = model_info['optimal_params_all'][i]
        change_pct = (optimized - original) / abs(original) * 100
        comparison_data.append({
            'Parameter': name,
            'Standard': original,
            'Optimized': optimized,
            'Change (%)': change_pct
        })
    
    param_df = pd.DataFrame(comparison_data)
    print(param_df.to_string(index=False, float_format=lambda x: f'{x:.5f}' if abs(x) < 100 else f'{x:.2f}'))

# Section 3: PERFORMANCE COMPARISON
print("\n3. PERFORMANCE COMPARISON (Test Set):")
print("-" * 60)

if 'df_test_eval' in globals():
    baseline_mae = df_test_eval['Error_Baseline'].abs().dropna().mean()
    nc_mae = df_test_eval['Error_ML_nc'].abs().dropna().mean()
    
    print(f"Standard SRK/T (nc=1.3330):")
    print(f"  MAE: {baseline_mae:.3f} D")
    
    print(f"\nOptimized nc only (nc={model_info['optimal_nc']:.5f}):")
    print(f"  MAE: {nc_mae:.3f} D")
    print(f"  Improvement: {(baseline_mae - nc_mae)/baseline_mae * 100:.1f}%")
    
    if 'Error_ML_Full' in df_test_eval.columns:
        full_mae = df_test_eval['Error_ML_Full'].abs().dropna().mean()
        print(f"\nFully optimized SRK/T (all parameters):")
        print(f"  MAE: {full_mae:.3f} D")
        print(f"  Improvement: {(baseline_mae - full_mae)/baseline_mae * 100:.1f}%")
    
    if 'Error_Final' in df_test_eval.columns:
        ensemble_mae = df_test_eval['Error_Final'].abs().dropna().mean()
        print(f"\nML + Ensemble:")
        print(f"  MAE: {ensemble_mae:.3f} D")
        print(f"  Improvement: {(baseline_mae - ensemble_mae)/baseline_mae * 100:.1f}%")

# Section 4: FORMULA IMPLEMENTATION
print("\n4. FORMULA IMPLEMENTATION:")
print("-" * 60)

# For nc-only optimization
print("\nA. Simple Implementation (nc-only optimization):")
print("   Just change nc from 1.333 to {:.5f} in standard SRK/T".format(model_info['optimal_nc']))

# For full optimization
if 'optimal_params_all' in model_info:
    print("\nB. Full Implementation (all parameters optimized):")
    print("""
def calculate_SRKT_FacoDMEK_Full(AL, K, A_const):
    '''
    Fully optimized SRK/T formula for FacoDMEK eyes
    
    Parameters:
    -----------
    AL : float - Axial Length in mm
    K : float - Average keratometry in diopters
    A_const : float - A-constant for the IOL
    
    Returns:
    --------
    float - Predicted IOL power for emmetropia
    '''
    # Optimized constants
    nc = {:.5f}         # corneal refractive index
    cw_const1 = {:.5f}  # corneal width constant 1
    cw_const2 = {:.5f}  # corneal width constant 2
    cw_const3 = {:.5f}  # corneal width constant 3
    acd_const1 = {:.5f} # ACD constant 1
    acd_const2 = {:.5f} # ACD constant 2
    offset_const = {:.5f} # offset constant
    rethick_const1 = {:.5f} # retinal thickness constant 1
    rethick_const2 = {:.5f} # retinal thickness constant 2
    
    # Standard constants (unchanged)
    na = 1.336  # aqueous/vitreous refractive index
    
    # Corneal radius
    r = 337.5 / K
    
    # Axial length correction
    if AL > 24.2:
        LCOR = -3.446 + 1.716 * AL - 0.0237 * AL**2
    else:
        LCOR = AL
    
    # Corneal width
    Cw = cw_const1 + cw_const2 * LCOR + cw_const3 * K
    
    # Corneal height
    H = r - sqrt(r**2 - (Cw**2 / 4))
    
    # ACD calculation
    ACDconst = acd_const1 * A_const - acd_const2
    offset = ACDconst - offset_const
    ACDest = H + offset
    
    # Retinal thickness
    RETHICK = rethick_const1 - rethick_const2 * AL
    LOPT = AL + RETHICK
    
    # IOL power calculation
    ncm1 = nc - 1
    IOL = (1000 * na * (na * r - ncm1 * LOPT)) / 
          ((LOPT - ACDest) * (na * r - ncm1 * ACDest))
    
    return IOL
""".format(*model_info['optimal_params_all']))

# Section 5: CLINICAL RECOMMENDATIONS
print("\n5. CLINICAL RECOMMENDATIONS:")
print("-" * 60)
print("""
1. For quick implementation: Use nc-only optimization
   - Simply change nc from 1.333 to {:.5f}
   - Provides {:.1f}% improvement

2. For maximum accuracy: Use fully optimized parameters
   - Implement all parameter changes
   - Provides {:.1f}% improvement

3. For research settings: Consider ensemble approach
   - Combines optimized formula with ML corrections
   - Provides {:.1f}% improvement

4. Important notes:
   - Use KERATOMETRY values (not biometry K)
   - Optimization specific to FacoDMEK eyes
   - Based on {} training cases
   - Validated on {} test cases
""".format(
    model_info['optimal_nc'],
    (baseline_mae - nc_mae)/baseline_mae * 100 if 'nc_mae' in locals() else 0,
    (baseline_mae - full_mae)/baseline_mae * 100 if 'full_mae' in locals() else 0,
    (baseline_mae - ensemble_mae)/baseline_mae * 100 if 'ensemble_mae' in locals() else 0,
    len(df_train_val),
    len(df_test)
))

# Section 6: EXAMPLE CALCULATIONS
print("\n6. EXAMPLE CALCULATIONS:")
print("-" * 60)

# Example case
AL_ex = 23.5
K_ex = 44.0
A_const_ex = 118.7

print(f"Example case: AL={AL_ex}mm, K={K_ex}D, A-constant={A_const_ex}")
print()

# Calculate with different methods
iol_standard = calculate_SRKT(AL_ex, K_ex, A_const_ex, nc=1.333)
iol_nc_opt = calculate_SRKT(AL_ex, K_ex, A_const_ex, nc=model_info['optimal_nc'])

print(f"Standard SRK/T:        {iol_standard:.2f} D")
print(f"Optimized nc only:     {iol_nc_opt:.2f} D (diff: {iol_nc_opt - iol_standard:+.2f} D)")

if 'optimal_params_all' in model_info:
    iol_full_opt = calculate_SRKT_ml(AL_ex, K_ex, A_const_ex, model_info['optimal_params_all'])
    print(f"Fully optimized:       {iol_full_opt:.2f} D (diff: {iol_full_opt - iol_standard:+.2f} D)")

# Section 7: SAVE FORMULA DETAILS
print("\n7. EXPORT FORMULA:")
print("-" * 60)

# Create export dictionary
export_data = {
    'formula_name': 'SRK/T-FacoDMEK-Optimized',
    'optimization_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
    'training_cases': len(df_train_val),
    'test_cases': len(df_test),
    'optimal_nc': float(model_info['optimal_nc']),
    'baseline_mae': float(baseline_mae) if 'baseline_mae' in locals() else None,
    'optimized_nc_mae': float(nc_mae) if 'nc_mae' in locals() else None,
}

if 'optimal_params_all' in model_info:
    export_data['optimal_params_all'] = [float(p) for p in model_info['optimal_params_all']]
    export_data['param_names'] = param_names
    export_data['full_optimized_mae'] = float(full_mae) if 'full_mae' in locals() else None

# Save as JSON
import json
with open('SRKT_FacoDMEK_optimized_formula.json', 'w') as f:
    json.dump(export_data, f, indent=2)

# Save human-readable version
with open('SRKT_FacoDMEK_optimized_formula.txt', 'w') as f:
    f.write("OPTIMIZED SRK/T FORMULA FOR FacoDMEK EYES\n")
    f.write("="*60 + "\n\n")
    f.write(f"Date: {export_data['optimization_date']}\n")
    f.write(f"Training cases: {export_data['training_cases']}\n")
    f.write(f"Test cases: {export_data['test_cases']}\n\n")
    
    f.write("NC-ONLY OPTIMIZATION:\n")
    f.write(f"  nc = {export_data['optimal_nc']:.5f} (standard: 1.33300)\n\n")
    
    if 'optimal_params_all' in model_info:
        f.write("FULL PARAMETER OPTIMIZATION:\n")
        for i, name in enumerate(param_names):
            f.write(f"  {name}: {model_info['optimal_params_all'][i]:.5f}\n")
    
    f.write(f"\nPERFORMANCE:\n")
    f.write(f"  Baseline MAE: {baseline_mae:.3f} D\n")
    f.write(f"  NC-optimized MAE: {nc_mae:.3f} D\n")
    if 'full_mae' in locals():
        f.write(f"  Fully optimized MAE: {full_mae:.3f} D\n")

print("\nFormula saved to:")
print("  - SRKT_FacoDMEK_optimized_formula.json (for implementation)")
print("  - SRKT_FacoDMEK_optimized_formula.txt (human-readable)")

print("\n" + "="*80)
print("DOCUMENTATION COMPLETE!")
print("="*80)